<a href="https://colab.research.google.com/github/fboldt/aulasann/blob/main/aula14b_hugging_face_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from datasets import load_dataset

ds = load_dataset("stanfordnlp/imdb")

README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
ds["train"][100]

{'text': "Terrible movie. Nuff Said.<br /><br />These Lines are Just Filler. The movie was bad. Why I have to expand on that I don't know. This is already a waste of my time. I just wanted to warn others. Avoid this movie. The acting sucks and the writing is just moronic. Bad in every way. The only nice thing about the movie are Deniz Akkaya's breasts. Even that was ruined though by a terrible and unneeded rape scene. The movie is a poorly contrived and totally unbelievable piece of garbage.<br /><br />OK now I am just going to rag on IMDb for this stupid rule of 10 lines of text minimum. First I waste my time watching this offal. Then feeling compelled to warn others I create an account with IMDb only to discover that I have to write a friggen essay on the film just to express how bad I think it is. Totally unnecessary.",
 'label': 0}

In [ ]:
base_model = "distilbert/distilbert-base-uncased-finetuned-sst-2-english"

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(base_model)

def tokenize_dataset(data):
    # Keys of the returned dictionary will be added to the dataset as columns
    return tokenizer(data["text"], padding="max_length", truncation=True)

dataset = ds.map(tokenize_dataset)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
from transformers import TFAutoModelForSequenceClassification
from tensorflow.keras.callbacks import ModelCheckpoint

# Load and compile our model
model = TFAutoModelForSequenceClassification.from_pretrained(base_model)

tf_dataset = model.prepare_tf_dataset(
    dataset["train"],
    batch_size=16,
    shuffle=True,
    tokenizer=tokenizer
)

tf_val_dataset = model.prepare_tf_dataset(
    dataset["validation"],  # ou "test" se não houver "validation"
    batch_size=16,
    shuffle=False,
    tokenizer=tokenizer
)

checkpoint_callback = ModelCheckpoint(
    filepath="melhor_modelo.h5",         # Caminho onde o modelo será salvo
    monitor="val_loss",                  # Métrica a ser monitorada
    save_best_only=True,                 # Salva apenas o melhor modelo
    save_weights_only=False,             # Salva o modelo completo (não só os pesos)
    mode="min",                          # Queremos minimizar a val_loss
    verbose=1                            # Mostra mensagens durante o salvamento
)

# Lower learning rates are often better for fine-tuning transformers
model.compile()  # No loss argument!

model.fit(
    tf_train_dataset,
    validation_data=tf_val_dataset,
    epochs=5,
    callbacks=[checkpoint_callback]
)

from tensorflow.keras.models import load_model

melhor_modelo = load_model("melhor_modelo.h5")

2025-07-07 23:33:30.061119: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751931210.226350      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751931210.277383      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

I0000 00:00:1751931224.746483      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.
I0000 00:00:1751931241.523433     126 service.cc:148] XLA service 0x7d3d16063a40 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1751931241.523907     126 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1751931241.592800     126 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1751

1562/1562 [==============================] - 834s 521ms/step - loss: 0.6986


In [ ]:
tf_test_dataset = model.prepare_tf_dataset(
    dataset["test"],
    batch_size=16,
    shuffle=False,
    tokenizer=tokenizer
)

In [ ]:
# Avaliar o modelo no conjunto de teste
results = model.evaluate(tf_test_dataset)
print("Resultados da avaliação:", results)

1563/1563 [==============================] - 279s 177ms/step - loss: 0.6939
Resultados da avaliação: 0.693935751914978


In [ ]:
from transformers import pipeline

clf_pipeline = pipeline(
    task="text-classification",
    model=model,
    tokenizer=tokenizer,
    return_all_scores=True  # Opcional: retorna scores para todas as classes
)

# Exemplo de uso
texts = [
    "This movie is excellent!",
    "The movie was horrible and boring."
]

results = clf_pipeline(texts)

for text, res in zip(texts, results):
    print(f"\nTexto: {text}")
    for label_score in res:
        print(f"  → {label_score['label']}: {label_score['score']:.4f}")

Device set to use 0
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(



Texto: This movie is excellent!
  → NEGATIVE: 0.5197
  → POSITIVE: 0.4803

Texto: The movie was horrible and boring.
  → NEGATIVE: 0.5197
  → POSITIVE: 0.4803


In [ ]:
model.save_pretrained("analise_sentimento_imdb_1epoca")
tokenizer.save_pretrained("analise_sentimento_imdb_1epoca")

('analise_sentimento_imdb_1epoca/tokenizer_config.json',
 'analise_sentimento_imdb_1epoca/special_tokens_map.json',
 'analise_sentimento_imdb_1epoca/vocab.txt',
 'analise_sentimento_imdb_1epoca/added_tokens.json',
 'analise_sentimento_imdb_1epoca/tokenizer.json')

In [ ]:
clf_pipeline = pipeline("text-classification", model="analise_sentimento_imdb_1epoca", tokenizer="analise_sentimento_imdb_1epoca")

Some layers from the model checkpoint at analise_sentimento_imdb_1epoca were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at analise_sentimento_imdb_1epoca and are newly initialized: ['dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use 0


In [ ]:
clf_pipeline("This is a great movie!")

[{'label': 'NEGATIVE', 'score': 0.5197456479072571}]